In [1]:
import nltk
import pandas as pd
import numpy as np
import os, sys
PATH = os.path.abspath("")[:-3]

In [ ]:
# SOLO HACER LA PRIMERA VEZ QUE SE TIRA
# Se va a abrir en una ventana aparte de python, le das a download todo y cuando acabe la cierras.
# Si todo ha salido guay devuelve True.
nltk.download()

# Preparar datos para predecir un usuario

In [7]:

# SCRAPPEAR UN USUARIO
from scrap import scrap_user

USER = "ComicGirlAshley"
tweets = scrap_user(USER)
tweets.to_csv(PATH+"data/example.csv", index=False) # Se guardan los tweets en un csv pero no hace falta lol.

In [ ]:
# CÁLCULO DEL FACTOR REP
from sklearn.feature_extraction.text import CountVectorizer
tweets = pd.read_csv(PATH+"data/example.csv") # Se puede leer el csv de antes.
contents = tweets["content"]
toks = []
sw = nltk.corpus.stopwords
stemmer = nltk.stem.snowball.EnglishStemmer()
# Tokenizar, stemmear, filtrar stop-words.
for content in contents.values:
    tk = nltk.word_tokenize(content)
    filtered_sentence = [w for w in tk if not w.lower() in sw.words('english')]
    toks.append([stemmer.stem(word) for word in filtered_sentence])
# Se cuenta número de repeticiones de palabras y se suman.
try:
    vect = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False, ngram_range=(1, 15), min_df=0.1).fit_transform(toks)
    factor_rep = vect.sum()
except:
    factor_rep = 0
print(factor_rep)

In [ ]:
# REPETICIÓN DE LAS FECHAS
from datetime import datetime
date = tweets["date"]
hours = []
for h in date.values:
    hours.append(pd.to_datetime(h).hour)
valores_finales=[]
tot= len(hours)
result = dict()
#contando el numero de repeticiones
for j in hours:
    if j not in result:
        result[j] = 0 
    result[j] += 1

#viendo el mayor valor y computando porcentaje
values=[]
mayor=0
for dato, valor in result.items():
    if valor > mayor:
        mayor= valor
hora_rep = mayor/tot
print(hora_rep)

In [10]:
# DATOS MEDIOS
avg_likes = np.average(tweets["likeCount"].values)
avg_retweets = np.average(tweets["retweetCount"].values)
avg_reply = np.average(tweets["replyCount"].values)
avg_quote = np.average(tweets["quoteCount"].values)

In [ ]:
# El usuario listo para predecir se guarda en user_data.
user_data = tweets[["verified", "followersCount", "friendsCount", "tweetsCount", "listedCount", "mediaCount"]].iloc[-1, :].values.tolist()
user_data.insert(0, factor_rep)
user_data.extend([avg_reply, avg_retweets, avg_likes, avg_quote, hora_rep, user_data[2]-user_data[3]])
print(user_data)
print(len(user_data))

# Preparacion train set

In [4]:
# Se leen los datos generados con model.py
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
training = pd.read_csv(PATH+"data/training_new.csv").sample(frac=1).reset_index(drop=True)
training["clase"].replace("bot",1, inplace= True)
training["clase"].replace("human",0, inplace= True)
training.pop("username")
label = training.pop("clase")
#se usaran para el estudio de los hiperparemtros dos funciones de normalización
scaler= StandardScaler()
data_s = scaler.fit_transform(training)
scaler_m= MinMaxScaler()
data_m = scaler.fit_transform(training)

## entrenamiento de modelos

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import xgboost as xgb


hiperparametros regresion

In [ ]:
data= data_s
x_train, x_test, y_train, y_test = train_test_split(data, label)

parameters= {"solver":["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]}
reg= LogisticRegression()

reg_n = GridSearchCV(reg, parameters, cv=5)
reg_n.fit(x_train, y_train)
print(reg_n.best_params_)
print(reg_n.best_score_)


regresion:

In [ ]:
data= data_s
skf= StratifiedKFold(n_splits=4)
precision=[]
sens=[]
spec=[]
acu=[]

for train_index, test_index in skf.split(data, label):
    x_train= data[train_index]
    y_train = label[train_index]
    x_test= data[test_index]
    y_test = label[test_index]

    reg = LogisticRegression(solver= "sag").fit(x_train, y_train)
    y_prep = reg.predict(x_test)
    acu.append(reg.score(x_test, y_test))
    tn, fp, fn, tp= metrics.confusion_matrix(y_test, y_prep).ravel()
    precision.append(tp/(tp+fp))
    sens.append(tp/(tp+fn))
    spec.append(tn/(tn+fp))

print("acurracy", np.mean(acu))
print("precision" ,np.mean(precision))
print("sens", np.mean(sens))
print("spec", np.mean(spec))

'''
acurracy 0.7101990049751243
precision 0.6736761164619097
sens 0.9750772953897953
spec 0.34062954531082024'''

hiperparametros xgb

In [ ]:
data= data_m
x_train, x_test, y_train, y_test = train_test_split(data, label)

parameters= {"booster": ["gbtree", "gblinear", "dart"], "nthread": range(1,32,2)}
xgb_= xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

xgb_n = GridSearchCV(xgb_, parameters, cv=5)
xgb_n.fit(x_train, y_train)
print(xgb_n.best_params_)
print(xgb_n.best_score_)

xgb

In [ ]:
data= data_m
skf= StratifiedKFold(n_splits=4)
precision=[]
sens=[]
spec=[]
acu=[]

for train_index, test_index in skf.split(data, label):
    x_train= data[train_index]
    y_train = label[train_index]
    x_test= data[test_index]
    y_test = label[test_index]

    xgb_= xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", booster= "gbtree", nthread= 1).fit(x_train,y_train)
    y_prep = xgb_.predict(x_test)
    acu.append(xgb_.score(x_test, y_test))
    tn, fp, fn, tp= metrics.confusion_matrix(y_test, y_prep).ravel()
    precision.append(tp/(tp+fp))
    sens.append(tp/(tp+fn))
    spec.append(tn/(tn+fp))

print("acurracy", np.mean(acu))
print("precision" ,np.mean(precision))
print("sens", np.mean(sens))
print("spec", np.mean(spec))

'''
acurracy 0.8146766169154229
precision 0.8303567912649471
sens 0.8569367553742553
spec 0.7556994245241258
'''

hiperparametro neuronas

In [ ]:
data= data_m
x_train, x_test, y_train, y_test = train_test_split(data, label)

parameters= {"activation": ["identity", "logistic", "tanh", "relu"], "learning_rate":["constant,invscaling", "adaptive"]}
ne= MLPClassifier(max_iter=10000)

ne_n = GridSearchCV(ne, parameters, cv=5)
ne_n.fit(x_train, y_train)
print(ne_n.best_params_)
print(ne_n.best_score_)


neuronas

In [ ]:
data= data_m
skf= StratifiedKFold(n_splits=4)
precision=[]
sens=[]
spec=[]
acu=[]

for train_index, test_index in skf.split(data, label):
    x_train= data[train_index]
    y_train = label[train_index]
    x_test= data[test_index]
    y_test = label[test_index]

    ne = MLPClassifier(activation="relu",learning_rate="adaptive", max_iter=10000).fit(x_train,y_train)
    y_prep = ne.predict(x_test)
    acu.append(ne.score(x_test, y_test))
    tn, fp, fn, tp= metrics.confusion_matrix(y_test, y_prep).ravel()
    precision.append(tp/(tp+fp))
    sens.append(tp/(tp+fn))
    spec.append(tn/(tn+fp))

print("acurracy", np.mean(acu))
print("precision" ,np.mean(precision))
print("sens", np.mean(sens))
print("spec", np.mean(spec))

'''
acurracy 0.7645107794361525
precision 0.7549646432499513
sens 0.8832718207718208
spec 0.5988387718965409

'''

probar usuario

In [12]:
#transformando los datos del scrapeado a la normalizacion necesaria y se selecciona el modelo querido

model= xgb_
scaler_m= MinMaxScaler()
user_d=np.array(user_data)
user_array= user_d.reshape(-1, 1)
user_array = scaler.fit_transform(user_array)

final= []
for i in range(0,13):
    final.append(float(user_array[i]))


In [ ]:
# Imprime la probabilidad de que el usuario preparado antes sea bot.
print(model.predict_proba(np.array([final]))[0])
result= model.predict(np.array([final]))[0]
if result==1:
    print("bot")
else:
    print("human")
